# QDutch

A reimagined card game by Francis Blais, Vincent Girouard, Olivier Romain and Marius Trudeau for iQuCodeFest 2025.

## Original Game Mechanics

QDutch is a reinterpretation of the Dutch card game. The rules of this game are simple.

- Each player (2 - 4) starts with four cards and can only look at two of them. The players then hide all of their cards and the game can start. 

- The goal of the game is to have the smallest sum of cards. 
One by one, each player picks up one card from a deck and look at it. If the player wants, he can replace any of his cards by that new card in order to lower his total sum. Otherwise, he can get rid of this card. 

- Some cards, when picked up from the deck, have special effects, e.g. look at a card, change a card with another player, etc. 

- At the beginning of his turn, if a player thinks he has the smallest hand, he calls *"Dutch"*. The game end after the turn and every player reveals his four cards. The player with the smallest sum wins the game.

## Quantumized Game Mechanics

**QDutch** changes some of those game mechanics to include quantum effects in the game. 

### 1. Player Cards

Each card of a player consists of a 3-qubit state. The *quantity* associated to this card is the measured value of the state in the computational basis (in binary). Because of that, each card can go from 0 to 7 points. At the beginning of the game, each player is provided with four states of the 3-qubit computational basis. Only two of them are known by the player. These states can be known exactly since they are eigenstates in the computational basis.

#### Code implementation

Each player card is implemented in a `PlayerSlot` class. These classes are initialized with a `Qiskit` quantum circuit, which represent the quantum state associated with the card.

In [1]:
from player_slot_class import PlayerSlot
from card_generator import generate_state

# Generate a state between 0 and 7 (random 3 qubit state in the computational basis)
state_index = generate_state()
print(f"{state_index = }\n")

# Create the Player Slot object with the generated state
state = PlayerSlot()
state.set_state(state_index)

# PlayerSlot contains a quantum circuit that prepares the state
print(state.qc)


state_index = 3

     ┌──────────────────────────────┐
q_0: ┤0                             ├
     │                              │
q_1: ┤1 Initialize(0,0,0,1,0,0,0,0) ├
     │                              │
q_2: ┤2                             ├
     └──────────────────────────────┘
c: 3/════════════════════════════════
                                     


### 2. Deck of cards

The deck of card in the middle of the table is also quantum. This deck contains three types of cards: **States**, **Action** and **Measurement**. Those cards are generated classically with predefined probabilities. Those cards are generated with the `card_generator` module.

a) States

State cards are 3 qubit quantum states expressed in the computationnal basis. When a player picks up a state card, he can change it with any of his cards. This create a new cards with zero uncertainty.

#### Code Implementation

The game is handled into a `QDutch` class. In this example, we show how a player can change one of his cards with a new card drawn from the deck.

In [13]:
from game_class import QDutch
from card_generator import Card, generate_state

# Initialize the game and add 2 players. Select the first player.
game = QDutch()
game.start_game(2)
game.init_routine()
player = game.players[0]

# Print the initial hand of the player
print("Initial Hand:")
print(player.hand[0].measure_all(), end="\n\n")

# Draw a state card from the deck
card = Card(type="State", data=generate_state())
print("State Card:")
print(card, end="\n\n")

# Apply the state card to the player's hand at the 1st position
game.apply_state_card(0, card)
print("New Card:")
print(player.hand[0].measure_all())


Initial Hand:
2

State Card:
Card(type='State', data=3)

New Card:
3


b) Action

Action cards are operators (quantum gates) that can be applied on state cards. When a player picks up an action card from the deck, he can apply it on any cards on the table (his cards and other players cards). The state the change according to the operator applied.

Operators can be single-qubit gates ($X$, $H$, $Z$, $S$) that apply on one of the 3 qubits of the states. They can also be two-qubit gates ($CNOT$, $SWAP$). 

Single-qubit gates can be used for instance to flip a bit or to generate quantum superposition. Two-qubit gates can be used to create entanglement in a state.




In [51]:
from player_class import Player
from game_class import QDutch
from card_generator import Card, generate_operator

game = QDutch()
game.start_game(2)
game.init_routine()
player = game.players[0]

print("Initial Hand:")
print(player.hand[0].measure_all())
print()

card = Card(type="Operator", data=generate_operator())
print("State Card:")
print(card)
print()

game.apply_operator_card(0, 0, card)
print("New Card:")
print(player.hand[0].measure_all())

Initial Hand:
4

State Card:
Card(type='Operator', data=['I', 'X', 'I'])

New Card:
6


c) Measurements

Measurement cards can measure a specified qubit on any chosen card on the table. Measurements collapse a state card and remove uncertainty on a qubit in a state. It also introduce probabilities if the measured qubit is in a superposition state. 

In [58]:
from player_class import Player
from game_class import QDutch
from card_generator import Card, generate_measurement

game = QDutch()
game.start_game(2)
game.init_routine()
player = game.players[0]

print("Initial Hand:")
print(player.hand[0].measure_all())
print()

card = Card(type="Measurement", data=generate_measurement())
print("State Card:")
print(card)
print()

bit_value = game.apply_operator_card(0, 0, card)
print("Bit value:")
print(bit_value)

Initial Hand:
5

State Card:
Card(type='Measurement', data=2)

Bit value:
1


### 3. End Game

At the end of the game, when a player calls *"Dutch"*, all cards on the table are measured (the three qubit are measured for each cards), which reveals the value of each card. The measured value can be probabilitic if the state is in superposition. These values are then added up to generate the score of the player. The player with the lowest score wins the game.

In [8]:
# Suppose a player have a card with a specific state:

state = PlayerSlot()
state.apply_operator(["X", "I", "I"])  # Apply X gate to the first qubit
state.apply_operator(["I", "I", "H"])  # Apply X gate to the second qubit
state.apply_operator(["C", "X", "I"])  # Apply H gate to the third qubit
print(state.qc)

# At the end of the game, all the players' cards are measured:
card_value = state.measure_all()
print(f"Card value: {card_value}")

     ┌───┐     
q_0: ┤ X ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
     ┌───┐└───┘
q_2: ┤ H ├─────
     └───┘     
c: 3/══════════
               
Card value: 3
